## Task 4: Geographical Analysis

In [10]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
from functools import partial, reduce

In [14]:
# Collect water scarcity data from the Food and Agriculture Organization (FAO)
master_csv = pd.read_csv('Resources/master_data.csv')

#filter by selected date range 1980 - 2015
master_dates_df = master_csv[master_csv['Year'].between(1980,2015)]

master_dates_df

,Unnamed: 0,Year,Country,HDI,Pop Density,Wtr Stress,Tot Wtr Resource,FreshW Wdrl,Total Population,Total Withdrawl,Wtr Withdrawl,Event Number,Conflict Name,Hosility Level,Target Country,Homicides,Battle Deaths,Theft & Vandalism,PSI
24314,24314,1980.0,Afghanistan,0.0,19.126047,46.223790,0.0,17.121292,12486.631,17.121292,1371.169827,30.0,No Conflict,4.0,Pakistan,0.000000,0,0.0,0.000000
24315,24315,1980.0,Afghanistan,0.0,19.126047,46.223790,0.0,17.121292,12486.631,17.121292,1371.169827,33.0,No Conflict,4.0,Pakistan,0.000000,0,0.0,0.000000
24316,24316,1980.0,Afghanistan,0.0,19.126047,46.223790,0.0,17.121292,12486.631,17.121292,1371.169827,45.0,No Conflict,2.0,Pakistan,0.000000,0,0.0,0.000000
24317,24317,1980.0,Afghanistan,0.0,19.126047,46.223790,0.0,17.121292,12486.631,17.121292,1371.169827,30.0,No Conflict,4.0,Pakistan,0.000000,0,0.0,0.000000
24318,24318,1980.0,Afghanistan,0.0,19.126047,46.223790,0.0,17.121292,12486.631,17.121292,1371.169827,33.0,No Conflict,4.0,Pakistan,0.000000,0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49427,49427,2015.0,World,0.0,0.000000,18.060000,0.0,3856.185980,7432664.552,3992.399524,537.142433,0.0,No Conflict,0.0,0,5.911969,0,0.0,0.000000
49428,49428,2015.0,Yemen,0.0,54.011677,169.761905,0.0,3.565000,28516.545,3.565000,125.015145,0.0,No Conflict,0.0,0,0.000000,0,0.0,0.000000
49429,49429,2015.0,"Yemen, Rep.",0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.0,No Conflict,0.0,0,0.000000,7032,0.0,-2.668016
49430,49430,2015.0,Zambia,0.0,21.589176,2.835498,0.0,1.572000,16248.230,1.572000,96.749000,0.0,No Conflict,0.0,0,5.249803,0,0.0,0.144710


,Unnamed: 0,Year,Country,HDI,Pop Density,Wtr Stress,Tot Wtr Resource,FreshW Wdrl,Total Population,Total Withdrawl,Wtr Withdrawl,Event Number,Conflict Name,Hosility Level,Target Country,Homicides,Battle Deaths,Theft & Vandalism,PSI
24314,24314,1980.0,Afghanistan,0.0,19.126047,46.223790,0.0,17.121292,12486.631,17.121292,1371.169827,30.0,No Conflict,4.0,Pakistan,0.000000,0,0.0,0.000000
24315,24315,1980.0,Afghanistan,0.0,19.126047,46.223790,0.0,17.121292,12486.631,17.121292,1371.169827,33.0,No Conflict,4.0,Pakistan,0.000000,0,0.0,0.000000
24316,24316,1980.0,Afghanistan,0.0,19.126047,46.223790,0.0,17.121292,12486.631,17.121292,1371.169827,45.0,No Conflict,2.0,Pakistan,0.000000,0,0.0,0.000000
24317,24317,1980.0,Afghanistan,0.0,19.126047,46.223790,0.0,17.121292,12486.631,17.121292,1371.169827,30.0,No Conflict,4.0,Pakistan,0.000000,0,0.0,0.000000
24318,24318,1980.0,Afghanistan,0.0,19.126047,46.223790,0.0,17.121292,12486.631,17.121292,1371.169827,33.0,No Conflict,4.0,Pakistan,0.000000,0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49427,49427,2015.0,World,0.0,0.000000,18.060000,0.0,3856.185980,7432664.552,3992.399524,537.142433,0.0,No Conflict,0.0,0,5.911969,0,0.0,0.000000
49428,49428,2015.0,Yemen,0.0,54.011677,169.761905,0.0,3.565000,28516.545,3.565000,125.015145,0.0,No Conflict,0.0,0,0.000000,0,0.0,0.000000
49429,49429,2015.0,"Yemen, Rep.",0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.0,No Conflict,0.0,0,0.000000,7032,0.0,-2.668016
49430,49430,2015.0,Zambia,0.0,21.589176,2.835498,0.0,1.572000,16248.230,1.572000,96.749000,0.0,No Conflict,0.0,0,5.249803,0,0.0,0.144710


### Cleanup the Militarized Interstate Events (MIE) csv file

In [3]:
# Copy the dataframe with only the columns we want 
mie_df = mie_csv[['styear', 'ccode1', 'eventnum', 'micnum', 'hostlev', 'ccode2']].copy()

# Create a dictionary for the country codes and their names and the confrontation codes and their name
code_to_country = pd.Series(COW_Country_Codes.StateNme.values, index=COW_Country_Codes.CCode).to_dict()
conflict_name = pd.Series(micnames.micname.values, index= micnames.micnum).to_dict()

# Map the country codes to their names from the dictionary and replace
mie_df['ccode1'] = mie_df['ccode1'].map(code_to_country)
mie_df['ccode2'] = mie_df['ccode2'].map(code_to_country)
mie_df['micnum'] = mie_df['micnum'].map(conflict_name)

# Rename columns headers
mie_df_clean = mie_df.rename(columns={'styear': 'Year',
                                'ccode1': 'Country',
                                'ccode2': 'Target Country',
                                'eventnum': 'Event Number',
                                'micnum': 'Conflict Name',
                                'hostlev': 'Hosility Level'
                                })
# Display the clean dataframe
mie_df_clean.head()

,Year,Country,Event Number,Conflict Name,Hosility Level,Target Country
0,1902,United States of America,1,Alaska Boundary Dispute (1902),3,United Kingdom
1,1913,Austria-Hungary,1,Serbian and Austro-Hungarian Fighting over Alb...,2,Yugoslavia
2,1946,Albania,2,British Attempts to Pass the Albanian Corfu Ch...,4,United Kingdom
3,1946,United Kingdom,3,British Attempts to Pass the Albanian Corfu Ch...,3,Albania
4,1946,United Kingdom,4,British Attempts to Pass the Albanian Corfu Ch...,3,Albania


### Cleanup the AQUASTAT csv

In [4]:
# Copy the dataframe with only the columns we want 
aqua_df = aqua_csv[['Year', 'Area', 'Variable', 'Value', 'Unit']].copy()

# Rename column header
aqua_df = aqua_df.rename(columns={'Area': 'Country'})

# Replace country with dictionary values
aqua_df['Country'] = aqua_df['Country'].replace(code_to_country)

#### Create columns for all variables in the export and create a new dataframe with new column values

In [5]:
# Create dataframe for Human Capital Index (max value = 1)
hdi_df = aqua_df.loc[aqua_df['Variable'] == 'Human Development Index (HDI)']
hdi_df = hdi_df.rename(columns={'Value': 'HDI'}).drop(columns=['Variable', 'Unit'])

In [6]:
# Create dataframe for Pop Density (ppl/km2)
pop_dens_df = aqua_df.loc[aqua_df['Variable'] == 'Population density']
pop_dens_df = pop_dens_df.rename(columns={'Value': 'Pop Density'}).drop(columns=['Variable', 'Unit'])

In [7]:
# Create dataframe for Wtr Stress %
wstress_df = aqua_df.loc[aqua_df['Variable'] == 'SDG 6.4.2. Water Stress']
wstress_df = wstress_df.rename(columns={'Value': 'Wtr Stress'}).drop(columns=['Variable', 'Unit'])

In [8]:
# Create dataframe for Total exploitable water resources (1b m3/yr)
tw_res_df = aqua_df.loc[aqua_df['Variable'] == 'Total exploitable water resources']
tw_res_df = tw_res_df.rename(columns={'Value': 'Tot Wtr Resource'}).drop(columns=['Variable', 'Unit'])

In [9]:
# Create dataframe for Total freshwater withdrawal 1b m3/yr)
tfw_wdrl_df = aqua_df.loc[aqua_df['Variable'] == 'Total freshwater withdrawal']
tfw_wdrl_df = tfw_wdrl_df.rename(columns={'Value': 'FreshW Wdrl'}).drop(columns=['Variable', 'Unit'])

In [10]:
# Create dataframe for Total Population (1000ppl)
tpop_df = aqua_df.loc[aqua_df['Variable'] == 'Total population']
tpop_df = tpop_df.rename(columns={'Value': 'Total Population'}).drop(columns=['Variable', 'Unit'])

In [11]:
# Create dataframe for Total Water Withdrawl (ppl/km2)
twdrl_df = aqua_df.loc[aqua_df['Variable'] == 'Total water withdrawal']
twdrl_df = twdrl_df.rename(columns={'Value': 'Total Withdrawl'}).drop(columns=['Variable', 'Unit'])

In [12]:
# Create dataframe for Total water withdrawal per capita (m3/ppl/yr)
tw_wdrl_pc_df = aqua_df.loc[aqua_df['Variable'] == 'Total water withdrawal per capita']
tw_wdrl_pc_df = tw_wdrl_pc_df.rename(columns={'Value': 'Wtr Withdrawl'}).drop(columns=['Variable', 'Unit'])

In [13]:
# Make a list of dataframes
aqua_df_lst = [hdi_df, pop_dens_df, wstress_df, tw_res_df, tfw_wdrl_df, tpop_df, twdrl_df, tw_wdrl_pc_df]

# Create a clean dataframe for water data 
aqua_df_clean = reduce(lambda left,right: pd.merge(left,right,on=['Year', 'Country'],how='outer'), aqua_df_lst)

# Display the clean dataframe
aqua_df_clean.head()

,Year,Country,HDI,Pop Density,Wtr Stress,Tot Wtr Resource,FreshW Wdrl,Total Population,Total Withdrawl,Wtr Withdrawl
0,1967.0,Afghanistan,NaN,15.332583,NaN,NaN,NaN,10010.030,NaN,NaN
1,1967.0,Albania,NaN,74.171930,NaN,13.0,NaN,2132.443,NaN,NaN
2,1967.0,Algeria,NaN,5.414997,NaN,7.9,NaN,12897.115,NaN,NaN
3,1967.0,Andorra,NaN,33.500000,NaN,NaN,NaN,15.745,NaN,NaN
4,1967.0,Angola,NaN,4.674343,NaN,NaN,NaN,5827.503,NaN,NaN


### Cleanup World Bank Group Crime csv

#### Create columns for each series name in the export and create a dataframe with all

In [14]:
# Filter out the homicides per 100k people
hom_df = crime_csv.loc[crime_csv['Series Name'] == 'Intentional homicides (per 100,000 people)'].set_index('Country Name')

# Use stack to drop unused columns and transpose the date columns into one columns
hom_df = hom_df.drop(columns=['Series Name', 'Series Code', 'Country Code']).stack().reset_index()

# Rename the column headers
hom_df = hom_df.rename(columns={'Country Name': 'Country',
                               'level_1': 'Year',
                                0: 'Homicides'
                               }).replace('..', 0)

In [15]:
# Filter out the total number of battle related people deaths
btl_dth_df = crime_csv.loc[crime_csv['Series Name'] == 'Battle-related deaths (number of people)'].set_index('Country Name')

# Use stack to drop unused columnstranspose the date columns into one columns
btl_dth_df = btl_dth_df.drop(columns=['Series Name', 'Series Code', 'Country Code']).stack().reset_index()

# Rename the column headers and replace null values ".." with 0
btl_dth_df = btl_dth_df.rename(columns={'Country Name': 'Country',
                                        'level_1': 'Year',
                                        0: 'Battle Deaths'
                                       }).replace('..', 0)

In [16]:
# Filter out the total % of annual sales firm losses due to theft and vandalism
theft_df = crime_csv.loc[crime_csv['Series Name'] == 'Losses due to theft and vandalism (% of annual sales for affected firms)'].set_index(
                                                                                                                                'Country Name')

# Use stack to drop unused columns and transpose the date columns into one columns
theft_df = theft_df.drop(columns=['Series Name', 'Series Code', 'Country Code']).stack().reset_index()

# Rename the column headers
theft_df = theft_df.rename(columns={'Country Name': 'Country',
                                        'level_1': 'Year',
                                        0: 'Theft & Vandalism'
                                       }).replace('..', 0)

In [17]:
# Make a list of dataframes
crime_df_lst = [hom_df, btl_dth_df, theft_df]

# Create a clean dataframe for water data 
crime_df_clean = reduce(lambda left,right: pd.merge(left,right,on=['Year', 'Country'],how='outer'), crime_df_lst)

# Replace country with dictionary values
crime_df_clean['Country'] = crime_df_clean['Country'].replace(code_to_country)

# Grab the first four values of the year column and convert to an integer
crime_df_clean['Year'] = crime_df_clean['Year'].str[0:4].astype(int)

# Display the clean dataframe
crime_df_clean.tail()

,Country,Year,Homicides,Battle Deaths,Theft & Vandalism
13295,West Bank and Gaza,2023,0,0,0
13296,World,2023,0,0,5.7
13297,"Yemen, Rep.",2023,0,0,0
13298,Zambia,2023,0,0,0
13299,Zimbabwe,2023,0,0,0


### Cleanup World Bank Group Political Stability csv

In [18]:
# Create stability dataframe from csv: set country as index, drop unused columns, stack the remaining columns, and reset the index
stability_df = stability_csv.set_index('Country Name').drop(columns=['Series Name', 'Series Code', 'Country Code']).stack().reset_index()

# Rename the column headers and replace null values ".." with 0
stability_df_clean = stability_df.rename(columns={'Country Name': 'Country',
                                                  'level_1': 'Year',
                                                  0: 'PSI'
                                                 }).replace('..', 0)

# Grab the first four values of the year column and convert to an integer
stability_df_clean['Year'] = stability_df_clean['Year'].str[0:4].astype(int)

# Display the clean dataframe
stability_df_clean.tail()

,Country,Year,PSI
5131,Zimbabwe,2018,-0.721038401126862
5132,Zimbabwe,2019,-0.943286120891571
5133,Zimbabwe,2020,-1.05272817611694
5134,Zimbabwe,2021,-0.954425930976868
5135,Zimbabwe,2022,-0.884499907493591


In [20]:
# Merge aquastat and conflicts dataframes by year and country
aqua_mie_df = pd.merge(aqua_df_clean, mie_df_clean, how='outer', on=['Year', 'Country'])

# Merge aquastat, conflicts and crime dataframes by year and country
aqua_mie_crime_df = pd.merge(aqua_mie_df, crime_df_clean, how='outer', on=['Year', 'Country'])

# Merge aquastat, conflicts, crime, and stability dataframes by year and country
master_df = pd.merge(aqua_mie_crime_df, stability_df_clean, how='outer', on=['Year', 'Country'])

# Fill the NaN under conflicts with no conflict
master_df['Conflict Name'] = master_df['Conflict Name'].fillna('No Conflict')
master_df = master_df.fillna(0)

# Display the final merged clean dataframe for analysis
master_df.tail()

,Year,Country,HDI,Pop Density,Wtr Stress,Tot Wtr Resource,FreshW Wdrl,Total Population,Total Withdrawl,Wtr Withdrawl,Event Number,Conflict Name,Hosility Level,Target Country,Homicides,Battle Deaths,Theft & Vandalism,PSI
53071,2023.0,West Bank and Gaza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Conflict,0.0,0,0,0,0,0
53072,2023.0,World,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Conflict,0.0,0,0,0,5.7,0
53073,2023.0,"Yemen, Rep.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Conflict,0.0,0,0,0,0,0
53074,2023.0,Zambia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Conflict,0.0,0,0,0,0,0
53075,2023.0,Zimbabwe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Conflict,0.0,0,0,0,0,0


In [21]:
# Write the new merged dataframe to a csv file
master_df.to_csv('Resources/master_data.csv')